In [1]:
#!pip install prefect[dask] dask-jobqueue
# prefect config set PREFECT_API_URL="https://ard-modeling-service.slac.stanford.edu/api" 

from dask.distributed import Client
from dask_jobqueue import SLURMCluster

# Simplify SLURMCluster parameters to basic working configuration

#SBATCH --qos=debug
#SBATCH --time=5
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=128
#SBATCH --constraint=cpu


cluster_kwargs = {
    "cores": 1,  
    "memory": "10GB",  # Correct spacing
    "shebang": "#!/bin/bash",
    #"account": "sanjeevc",
    "walltime": "00:10:00",
    "job_script_prologue": ["source ~/.bashrc"],
    # Only basic directives, add more as needed and test each step
    "job_extra_directives": ["-A m669","-q regular","--constraint=cpu"],
}
cluster = SLURMCluster(**cluster_kwargs)
print(cluster.job_script())


#!/bin/bash

#SBATCH -J dask-worker
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G
#SBATCH -t 00:10:00
#SBATCH -A m669
#SBATCH -q regular
#SBATCH --constraint=cpu
source ~/.bashrc
/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/bin/python -m distributed.cli.dask_worker tcp://128.55.64.31:34023 --name dummy-name --nthreads 1 --memory-limit 9.31GiB --nanny --death-timeout 60



In [ ]:
!dask-scheduler


/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/distributed/cli/dask_scheduler.py:142: FutureWarning: dask-scheduler is deprecated and will be removed in a future release; use `dask scheduler` instead
  warnings.warn(
2024-06-21 16:40:30,292 - distributed.scheduler - INFO - -----------------------------------------------
2024-06-21 16:40:30,662 - distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41873 instead
  warnings.warn(
2024-06-21 16:40:30,690 - distributed.scheduler - INFO - State start
2024-06-21 16:40:30,698 - distributed.scheduler - INFO - ----------------------------------

In [2]:
#!prefect config set PREFECT_API_URL="https://ard-modeling-service.slac.stanford.edu/api" 

In [3]:
!prefect config set PREFECT_API_URL="http://localhost:8000/api"


Set 'PREFECT_API_URL' to 'http://localhost:8000/api'.
Updated profile 'default'.


In [4]:
slurm_jobs = 1
cluster.scale(jobs=slurm_jobs)
client = Client(cluster)

In [5]:
print(client.dashboard_link)

http://128.55.64.31:8787/status


In [6]:
from prefect import flow, task
from prefect_dask import DaskTaskRunner

@flow(task_runner=DaskTaskRunner(address=client.scheduler.address))
def workflow(a: float, b: float) -> float:
    output1 = add.submit(a, b)
    output2 = mult.submit(output1, b)
    return output2

@task
def add(a: float, b: float) -> float:
    return a + b

@task
def mult(a: float, b: float) -> float:
    return a * b

In [7]:
output = workflow(1, 2)
print(output.result())

RuntimeError: Cannot create flow run. Failed to reach API at http://localhost:8000/api/.

# Temporary Dask Cluster
Some users may prefer to spin up a Dask cluster (i.e. Slurm job) for each indiviual @flow. This is also possible in Prefect. (Note that the example below will complain about an open port if you have already instantiated a Dask cluster above.)



from prefect import flow, task
from prefect_dask import DaskTaskRunner

@flow(task_runner=DaskTaskRunner(cluster_class=SLURMCluster, cluster_kwargs=cluster_kwargs))
def workflow(a: float, b: float) -> float:
    output1 = add.submit(a, b)
    output2 = mult.submit(output1, b)
    return output2

@task
def add(a: float, b: float) -> float:
    return a + b

@task
def mult(a: float, b: float) -> float:
    return a * b

workflow(1, 2).result()